In [1]:
import pandas as pd
# result_df = pd.read_csv("../../data/clean_data.500k.csv")
result_df = pd.read_csv("../../data/clean_data.csv")
display(result_df)

,Unnamed: 0,event_dt,screen,feature,action,device_id,session_id,device_vendor,device_model,device_type,os,age,gender,is_finish,is_churn,node_id
0,0,2025-09-29 10:20:27+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,875e50d28d26088a760866c1125ff124
1,1,2025-09-29 10:21:56+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,875e50d28d26088a760866c1125ff124
2,2,2025-09-29 10:22:01+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,5281fb229131fa372bd15589fed81bc7
3,3,2025-09-29 10:23:49+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,1,0,875e50d28d26088a760866c1125ff124
4,4,2025-09-29 10:20:17+03:00,Новая заявка,Выбор квартиры,Тап на квартиру,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,e9c334108c17d0d6014b73f196b775f4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360645,3360645,2025-10-09 16:18:31+03:00,Новое ОСС,Открытие экрана,Переход к экрану,123401,10000000010,Apple,iPhone 7 Plus,phone,iOS,51.0,Ж,1,0,18b0d595350748c3fd82491de6631219
3360646,3360646,2025-10-09 18:56:36+03:00,Еще,Открытие экрана,Переход к экрану,152365,10000000054,Apple,iPhone XR,phone,iOS,49.0,Ж,1,0,875e50d28d26088a760866c1125ff124
3360647,3360647,2025-10-09 14:41:10+03:00,Еще,Открытие экрана,Переход к экрану,99472,10000000098,Apple,iPhone 14 Pro Max,phone,iOS,34.0,Ж,0,0,875e50d28d26088a760866c1125ff124
3360648,3360648,2025-10-09 14:41:16+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',99472,10000000098,Apple,iPhone 14 Pro Max,phone,iOS,34.0,Ж,0,0,5281fb229131fa372bd15589fed81bc7


In [2]:
# Преобразование event_dt в datetime для корректной сортировки
result_df['event_dt'] = pd.to_datetime(result_df['event_dt'])

# Сортировка данных по устройству, сессии и времени
result_df = result_df.sort_values(['device_id', 'session_id', 'event_dt'])

# Создание столбца с следующей нодой в рамках одной сессии
result_df['next_node'] = result_df.groupby(['device_id', 'session_id'])['node_id'].shift(-1)

# Фильтрация: оставляем только строки, где есть следующая нода (переход)
transitions_df = result_df[['node_id', 'next_node']].dropna()
transitions_df.columns = ['source_id', 'target_id']

# Удаляем переходы из ноды в саму себя
transitions_df = transitions_df[transitions_df['source_id'] != transitions_df['target_id']]

# Оставляем только уникальные переходы
unique_transitions = transitions_df.drop_duplicates()

# Сохраняем в файл
unique_transitions.to_csv('../../data/links_graph.csv', index=False)

print(f"Таблица переходов сохранена в links.csv")
print(f"Количество уникальных переходов: {len(unique_transitions)}")

# Показать первые несколько строк для проверки
display(unique_transitions.head())


Таблица переходов сохранена в links.csv
Количество уникальных переходов: 3029


,source_id,target_id
614672,875e50d28d26088a760866c1125ff124,5281fb229131fa372bd15589fed81bc7
614671,5281fb229131fa372bd15589fed81bc7,ac398d9f0ba05d231b9d183feb595b12
614674,ac398d9f0ba05d231b9d183feb595b12,875e50d28d26088a760866c1125ff124
564401,875e50d28d26088a760866c1125ff124,c3617d19147db0f2c720ae56324619a3
934576,875e50d28d26088a760866c1125ff124,5d25ba67e2bc07adddc59f7a0e0baacd
